In [78]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

In [79]:
# URLs of pages to be scraped
nasa_mars_news_url = 'https://mars.nasa.gov/news/'
mars_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
mars_facts_url = 'http://space-facts.com/mars/'
mars_hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [80]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\ese12\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [81]:
def nasa():
        url = 'https://mars.nasa.gov/news/'
        browser.visit(url)
        time.sleep(2)

        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        news_ttl=[]
        for div in soup.find_all('div', attrs={'class':'content_title'}):
            news_ttl.append(div.find('a'))
        news_title=news_ttl[1].text

        news=soup.find('div', attrs={'class':'article_teaser_body'}).text
        return(news_title,news)

In [82]:
news_title,news_p=nasa()
print(f"News Title :{news_title}")
print(f"News Paragraph :{news_p}")

News Title :Could the Surface of Phobos Reveal Secrets of the Martian Past?
News Paragraph :The Martian moon Phobos orbits through a stream of charged atoms and molecules that flow off the Red Planet’s atmosphere, new research shows.


In [83]:
featured_image_url='https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html


In [84]:
facts_table = pd.read_html(mars_facts_url)
facts_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [86]:
browser.visit(mars_hemispheres_url)
#Creat lists to hold all of the urls and titles
hemispheres_image_urls = []
titles_list = []
for x in range(0, 4):
    #Go through the four page links and visit each one
#     browser.visit(links_list[x])
    #Print each link as its being processed
#     print(links_list[x])
    #Create soup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    #The jpg we are looking for is in the downloads class
    images = soup.find(class_='downloads')
    #Find the link tag and then get its link
    image = images.find('a')
    image_url= image['href']
    #Append that to the list
    hemispheres_image_urls.append(image_url)
    #Find title and append title to the list of titles
    titles = soup.find('h2', class_='title')
    title=titles.text
    title=title.strip('Enhanced')
    titles_list.append(title)
    #print each title and image url as they are processed
    print(title)
    print(image_url)
#Print the completed lists at the end    
print(hemispheres_image_urls)
print(len(hemispheres_image_urls))

AttributeError: 'NoneType' object has no attribute 'find'

In [87]:
browser.quit()